<a href="https://colab.research.google.com/github/SindhuraSriram/falcon-qlora-finetune/blob/main/falcon_rw_1b_dolly15k_qlora_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔧 Step 1: Set Up Environment
Install LLaMA-Factory and required libraries

In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0)

(True, 'Tesla T4')

In [ ]:
!nvidia-smi

Sun Jul 27 15:50:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install the necessary packages
!pip install -q transformers accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 827.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "tiiuae/falcon-rw-1b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="bfloat16"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token  # Falcon needs pad token set explicitly

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load Dolly instruction-tuning dataset
dataset = load_dataset("databricks/databricks-dolly-15k")

# Take a look at one sample
print(dataset["train"][0])

README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [ ]:
# Format the dataset into prompt-response text pairs
def format_prompt(sample):
    instruction = sample["instruction"]
    context = sample.get("context", "")
    response = sample["response"]

    if context:
        prompt = f"### Instruction:\n{instruction}\n\n### Context:\n{context}\n\n### Response:\n{response}"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{response}"

    return {"text": prompt}

# Apply formatting to the full dataset
train_data = dataset["train"].map(format_prompt)

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

max_length = 512  # Falcon 1B fits well with this limit on Colab

def tokenize(sample):
    return tokenizer(
        sample["text"],
        truncation=True,
        max_length=max_length,
        padding="max_length"
    )

# Tokenize all examples
tokenized_dataset = train_data.map(tokenize, batched=True, remove_columns=train_data.column_names)

# Use Hugging Face’s default LM data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# LoRA configuration for causal LM
lora_config = LoraConfig(
    r=8,                      # low-rank dimension
    lora_alpha=16,
    target_modules=["query_key_value"],  # specific to Falcon
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Wrap model with LoRA adapters
model = get_peft_model(model, lora_config)

# Print trainable params
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./falcon-qlora-dolly",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator
)

trainer.train()


/tmp/ipython-input-17-3719767079.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.352600
20,2.110400
30,2.092700
40,1.985900
50,2.101700
60,2.030700
70,2.070300
80,2.122900
90,2.038700
100,2.108500


TrainOutput(global_step=5631, training_loss=1.956278308545944, metrics={'train_runtime': 10050.3725, 'train_samples_per_second': 4.481, 'train_steps_per_second': 0.56, 'total_flos': 1.674173422113915e+17, 'train_loss': 1.956278308545944, 'epoch': 3.0})

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch

# Reuse tokenizer and bnb_config from earlier cells
model_name = "tiiuae/falcon-rw-1b"
checkpoint_path = "./falcon-qlora-dolly/checkpoint-5631"  # update to final checkpoint folder

# Load base Falcon model again in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load the final LoRA adapter (checkpoint number must match final)
lora_model = PeftModel.from_pretrained(base_model, checkpoint_path)
lora_model.eval()

# Inference prompt
prompt = """### Instruction:
What are the benefits of using solar energy?

### Response:"""

# Tokenize prompt
inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")

# Generate output
with torch.no_grad():
    outputs = lora_model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

# Decode and print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### Instruction:
What are the benefits of using solar energy?

### Response:
Solar energy is a clean, renewable energy source that is not limited by the sun's presence.  Solar energy is easily available and can be easily scaled up.  It is also not affected by weather, making it ideal for off-grid applications.  Solar energy can be a great option to help save money on electric bills.  Solar energy can also be a great source of income for those who are interested in this type of energy.  For those who are interested in the cost savings,


In [ ]:
from peft import PeftModel

# Reload adapter to avoid side-effects
lora_model = PeftModel.from_pretrained(base_model, "./falcon-qlora-dolly/checkpoint-5631")

# Push to your Hugging Face repo
lora_model.push_to_hub("falcon-rw-1b-dolly-qlora")

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SindhuraSriram/falcon-rw-1b-dolly-qlora/commit/393a5c529cf6592623f5fe9aab53b384b4113cfc', commit_message='Upload model', commit_description='', oid='393a5c529cf6592623f5fe9aab53b384b4113cfc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SindhuraSriram/falcon-rw-1b-dolly-qlora', endpoint='https://huggingface.co', repo_type='model', repo_id='SindhuraSriram/falcon-rw-1b-dolly-qlora'), pr_revision=None, pr_num=None)

In [ ]:
# ✅ Define test instructions
test_prompts = [
    "Explain the concept of cloud computing in simple terms.",
    "What are the key differences between supervised and unsupervised learning?",
    "How does the stock market work for beginners?",
    "What are black holes and how are they formed?",
    "Why is quantum computing considered revolutionary?",
    "Explain how solar panels generate electricity.",
    "How can I improve my time management as a college student?",
    "What are some good strategies for preparing for a coding interview?",
    "Write a short story about a robot who gains emotions.",
    "Generate 5 startup ideas based on remote work trends.",
    "If it takes 5 machines 5 minutes to make 5 widgets, how long would it take 100 machines to make 100 widgets?",
    "What will be the next number in the sequence: 2, 6, 12, 20, 30...?",
    "What are the pros and cons of using AI in education?",
    "How can we reduce plastic pollution at a global scale?",
    "Give me a step-by-step guide on how to train a small language model using LoRA."
]

# ✅ Run inference loop
for i, instruction in enumerate(test_prompts, 1):
    prompt = f"### Instruction:\n{instruction}\n\n### Response:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n--- Prompt {i} ---")
    print(f"Instruction: {instruction}")
    print("Response:")
    print(response.split("### Response:")[-1].strip())



--- Prompt 1 ---
Instruction: Explain the concept of cloud computing in simple terms.
Response:
Cloud computing, also known as cloud computing services or cloud computing as a service (CCaaS), is a model for enabling on-demand network-as-a-service (NaaS) applications and related infrastructure as a service (IaaS) by using the Internet as the infrastructure. Cloud computing services are delivered on-demand over an internet-based network, and consist of virtual machines (VMs) and other resources on cloud platforms managed by the cloud service provider (CSP) or cloud service provider's data center (CSPDC). Cloud computing services can include a range

--- Prompt 2 ---
Instruction: What are the key differences between supervised and unsupervised learning?
Response:
Supervised learning uses a model to predict the outcomes of a training set.
Unsupervised learning uses a model to predict the outcomes of a training set, but without specifying which features to use.
Supervised learning is typi